In [1]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

In [2]:
from openai_hackathon_wait.api.arxiv import arxiv_tool

In [3]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
import asyncio

arxiv_agent = Agent(
    name="Arxiv agent",
    instructions="You provide information about the papers that are related to the query.",
    tools=[arxiv_tool],
    model="gpt-4o-mini", 
)

triage_agent = Agent(
    name="Triage agent",
    instructions="You are a helpful assistant that finds the most relevant papers from Arxiv.",
    handoffs=[arxiv_agent],
    model="gpt-4o-mini",
)

In [4]:
async def main():
    result = await Runner.run(triage_agent, input="Find papers about soft decision trees in machine learning")
    print(result.final_output)

In [5]:
await main()

Here are some papers related to soft decision trees in machine learning:

1. **Rectified Decision Trees: Exploring the Landscape of Interpretable and Effective Machine Learning**
   - **Authors**: Yiming Li, Jiawang Bai, Jiawei Li, Xue Yang, Yong Jiang, Shu-Tao Xia
   - **Published**: 2020-08-21
   - **Summary**: This paper discusses the balance between interpretability and effectiveness in machine learning, particularly through the lens of decision trees.

2. **Analyzing Tree Architectures in Ensembles via Neural Tangent Kernel**
   - **Authors**: Ryuichi Kanoh, Mahito Sugiyama
   - **Published**: 2023-02-07
   - **Summary**: This work explores soft trees, a variant of decision trees that are characterized by their ability to update and adapt, using neural tangent kernels for analysis.

3. **Variational Boosted Soft Trees**
   - **Authors**: Tristan Cinquin, Tammo Rukat, Philipp Schmidt, Martin Wistuba, Artur Bekasov
   - **Published**: 2023-02-22
   - **Summary**: The authors focus o